In [1]:
import numpy as np
from os.path import isfile
import os
import random

In [2]:
np.random.seed(1234)
random.seed(1234)

In [1]:
import torch
from torch.nn import functional as F

In [15]:
from sklearn.metrics.pairwise import cosine_similarity

input1 = torch.randn(1, 3, 448, 448)
input2 = torch.randn(1, 3, 448, 448)

input1 = input1.reshape(input1.shape[0], -1)
input2 = input2.reshape(input2.shape[0], -1)
# output = F.cosine_similarity(input1, input2)
output = cosine_similarity(input1, input2)
print(output.shape)

(1, 1)


In [16]:
output = output.mean(axis=1)
print(output.shape)

(1,)


In [3]:
_base_dir = '/mnt/storage/fangyijie/HC18'

In [4]:
with open(_base_dir + "/ssl/train/labeled_data/data_50.list", "r") as f1:
    label_sample_list = f1.readlines()

In [5]:
with open(_base_dir + "/ssl/train/unlabeled_data/data_50.list", "r") as f1:
    unlabel_sample_list = f1.readlines()

In [6]:
print(len(label_sample_list))
print(len(unlabel_sample_list))

50
450


In [7]:
# size = 5
size = 10

In [17]:
from torch.utils.data import DataLoader
from dataloaders.dataset_fetalhead import BaseDataSets_HC18
from torchvision import transforms

tensor_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((448, 448), transforms.InterpolationMode.NEAREST),
])

db_train_al_unlabled = BaseDataSets_HC18(base_dir='/mnt/storage/fangyijie/HC18', split="train", num=10, islabeled=False, 
                                        isAL=True, alNum=0, stgrategy='RandomSampling',
                                        transform=None, default_transform=tensor_transforms)

/home/fangyijie/miniconda3/envs/sam_env/lib/python3.11/site-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.8' (you have '2.0.5'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [19]:
for idx, unlabeled_sample in enumerate(db_train_al_unlabled):
    print('test')
    unlabeled_img = unlabeled_sample['image']
    print('unlabeled_img.shape:', unlabeled_img.shape)
    break

test
unlabeled_img.shape: torch.Size([3, 448, 448])


In [9]:
unlabeled_idxs = np.array(range(len(db_train_al_unlabled)))
# unlabeled_idxs

In [ ]:
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def predict_prob(net, data):
    net.to(device)
    net.eval()
    probs = torch.zeros([len(data), 2, 448, 448])
    x_names = [None] * len(data)
    loader = DataLoader(data, batch_size=1, shuffle=False, num_workers=1)
    with torch.no_grad():
        # for x, y, idxs in loader:
        for i_batch, sampled_batch in enumerate(loader):
            volume_valbatch, label_valbatch = sampled_batch['image'], sampled_batch['label']
            x, y = volume_valbatch.cuda(), label_valbatch.cuda()
            prob, _ = net(x) # torch.Size([8, 2, 64, 64, 64])
            # prob = F.softmax(out, dim=1) # torch.Size([8, 2, 64, 64, 64])
            probs[i_batch] = prob.cpu() 
            x_names[i_batch] = sampled_batch['name'][0]
    return probs, np.array(x_names)

In [ ]:
from unext.archs import UNext as Unet2D
# UNeXt
test_model = Unet2D(in_chns=3, out_channels=2, img_size=448, num_classes=2).cpu()
# AL models 50
root_dir = '/mnt/storage/fangyijie/HC18'
test_model.load_state_dict(torch.load(f'{root_dir}/al/model/HC18/Semi_Unext_Swin_AL_10/Unext/model1_iter_20000.pth', map_location="cuda:0"))
# loaded_model = test_model.load(f'{root_dir}/al/model/HC18/Semi_Unext_Swin_AL_10/Unext', iternum=20000)

<All keys matched successfully>

In [12]:
probs, sample_names = predict_prob(net=test_model, data=db_train_al_unlabled)
uncertainties = probs.sum((1,2,3))#([N])

In [13]:
print(torch.max(uncertainties))
print(torch.min(uncertainties))

tensor(-2917.3013)
tensor(-92387.6328)


In [14]:
print(uncertainties.argmin().item())
print(uncertainties.argmax().item()) # get the index of the max uncertainty

143
276


In [17]:
sample_idx = unlabeled_idxs[uncertainties.sort()[1][:10].cpu().numpy()]

In [19]:
sample_idx

array([143,  95,  11,  91,  61, 404, 145, 408,  83, 377])

In [ ]:
sample_names[sample_idx]

In [26]:
new_unlabled = label_sample_list+sample_names[sample_idx].tolist()

In [27]:
with open(_base_dir + f"/ssl/train/labeled_data/EntropySampling/data_al_00_10.list", "w") as f:
    for line in (new_unlabled):
        f.write(f"{line}")

### AL Round 0

In [ ]:
num_round = 0

In [ ]:
random.seed(1234*(10))  # SSL
# random.seed(123*(size-1))  # Few-Shot
selected = random.sample(label_sample_list, size)

In [ ]:
unselected = list(set(label_sample_list) - set(selected))

In [ ]:
with open(_base_dir + f"/ssl/train/labeled_data/RandomSampling/data_al_{num_round}_{size}.list", "w") as f:
    for line in selected:
        f.write(f"{line}")

In [ ]:
with open(_base_dir + f"/ssl/train/unlabeled_data/RandomSampling/data_al_{num_round}_{size}.list", "w") as f:
    for line in (unselected+unlabel_sample_list):
        f.write(f"{line}")

In [ ]:
unselected = unselected+unlabel_sample_list

### AL Round 1

In [ ]:
num_round = 1

In [ ]:
random.seed(1234*(num_round))  # SSL
selected_1 = random.sample(unselected, size)

In [ ]:
unselected = list(set(unselected) - set(selected_1))

In [ ]:
len(unselected)

In [ ]:
selected = selected+selected_1

In [ ]:
with open(_base_dir + f"/ssl/train/labeled_data/RandomSampling/data_al_{num_round}_{size}.list", "w") as f:
    for line in selected:
        f.write(f"{line}")

In [ ]:
with open(_base_dir + f"/ssl/train/unlabeled_data/RandomSampling/data_al_{num_round}_{size}.list", "w") as f:
    for line in unselected:
        f.write(f"{line}")

### AL Round 2

In [ ]:
num_round = 2

In [ ]:
random.seed(1234*(num_round))  # SSL
selected_2 = random.sample(unselected, size)
unselected = list(set(unselected) - set(selected_2))

In [ ]:
selected = selected+selected_2

In [ ]:
print(len(selected))
print(len(unselected))

In [ ]:
with open(_base_dir + f"/ssl/train/labeled_data/RandomSampling/data_al_{num_round}_{size}.list", "w") as f:
    for line in selected:
        f.write(f"{line}")

In [ ]:
with open(_base_dir + f"/ssl/train/unlabeled_data/RandomSampling/data_al_{num_round}_{size}.list", "w") as f:
    for line in unselected:
        f.write(f"{line}")

### AL Round 3

In [ ]:
num_round = 3

In [ ]:
random.seed(1234*(num_round))  # SSL
selected_3 = random.sample(unselected, size)
unselected = list(set(unselected) - set(selected_3))

In [ ]:
selected = selected+selected_3

In [ ]:
print(len(selected))
print(len(unselected))

In [ ]:
with open(_base_dir + f"/ssl/train/labeled_data/RandomSampling/data_al_{num_round}_{size}.list", "w") as f:
    for line in selected:
        f.write(f"{line}")

In [ ]:
with open(_base_dir + f"/ssl/train/unlabeled_data/RandomSampling/data_al_{num_round}_{size}.list", "w") as f:
    for line in unselected:
        f.write(f"{line}")

### AL Round 4

In [ ]:
num_round = 4

In [ ]:
random.seed(1234*(num_round))  # SSL
selected_4 = random.sample(unselected, size)
unselected = list(set(unselected) - set(selected_4))

In [ ]:
selected = selected+selected_4

In [ ]:
print(len(selected))
print(len(unselected))

In [ ]:
with open(_base_dir + f"/ssl/train/labeled_data/RandomSampling/data_al_{num_round}_{size}.list", "w") as f:
    for line in selected:
        f.write(f"{line}")

In [ ]:
with open(_base_dir + f"/ssl/train/unlabeled_data/RandomSampling/data_al_{num_round}_{size}.list", "w") as f:
    for line in unselected:
        f.write(f"{line}")